# Description of the project

The Clear Taxi company has collected historical data on taxi orders at airports.
<br>In order to attract more drivers during the peak period, you need to predict the number of taxi orders for the next hour.
<br>It is necessary to build a model for such a prediction.
<br>The value of the RMSE metric on the test sample should not exceed 48.

# Description of data

The data is in the file `/datasets/taxi.csv`
<br>The number of orders is in the `num_orders` column

# Action plan

1. Load the data and resample it one hour at a time.
2. Analyze the data.
3. Train different models with different hyperparameters (make a test sample of 10% of the original data).
4. Check the data on the test sample and draw conclusions.

# Loading and resampling data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy

from statsmodels.tsa.seasonal import seasonal_decompose

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import lightgbm as lgb

from sklearn.metrics import mean_squared_error

pd.set_option('display.max_row',100)
pd.set_option('display.max_columns',100)

In [ ]:
df = pd.read_csv('/datasets/taxi.csv', index_col=[0], parse_dates=[0],sep=',')
print(df.info())
df.head()

In [ ]:
df.describe()

Ресемплируем данные по одному часу

In [ ]:
df = df.resample('1H').sum()
df.head()

# Data analysis

Check the indices for monotonicity

In [ ]:
df.index.is_monotonic

Total number of rows and number of missing values

In [ ]:
a = df['num_orders'].count()
b = df['num_orders'].isna().sum()
print(a)
print(b)
100*b/a

In [ ]:
plt.figure(figsize=(20, 6))
sns.lineplot(x=df.index,y=df['num_orders'])

Взглянем в разрезе 21-го дня

In [ ]:
df_1_7 = df['2018-03-01':'2018-03-07']
df_8_14 = df['2018-03-08':'2018-03-14']
df_15_21 = df['2018-03-15':'2018-03-21']

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(20, 10), sharey=True)
fig.suptitle('In the context of 21 days')

sns.lineplot(ax=axes[0], x=df_1_7.index,y=df_1_7['num_orders'])
axes[0].set_title('1-7')
axes[0].set_xlabel('')

sns.lineplot(ax=axes[1], x=df_8_14.index,y=df_8_14['num_orders'])
axes[1].set_title('8-14')
axes[1].set_xlabel('')

sns.lineplot(ax=axes[2], x=df_15_21.index,y=df_15_21['num_orders'])
axes[2].set_title('15-21')
axes[2].set_xlabel('')

Let's build the same graphs, but in the next months

In [ ]:
df_1_7 = df['2018-04-01':'2018-04-07']
df_8_14 = df['2018-04-08':'2018-04-14']
df_15_21 = df['2018-04-15':'2018-04-21']

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(20, 10), sharey=True)
fig.suptitle('In the context of 21 days')

sns.lineplot(ax=axes[0], x=df_1_7.index,y=df_1_7['num_orders'])
axes[0].set_title('1-7')
axes[0].set_xlabel('')

sns.lineplot(ax=axes[1], x=df_8_14.index,y=df_8_14['num_orders'])
axes[1].set_title('8-14')
axes[1].set_xlabel('')

sns.lineplot(ax=axes[2], x=df_15_21.index,y=df_15_21['num_orders'])
axes[2].set_title('15-21')
axes[2].set_xlabel('')

As you can see, the workload schedules during the week are similar.
<br>We continue to consider in more detail

Let's build a graph of trends and seasonality

In [ ]:
decomposed = seasonal_decompose(df)

In [ ]:
plt.figure(figsize=(20, 20))
plt.subplot(311)

decomposed.trend.plot(ax=plt.gca())
plt.title('Trend')

plt.subplot(312)
decomposed.seasonal.plot(ax=plt.gca()) 
plt.title('Seasonality')

plt.subplot(313)
decomposed.resid.plot(ax=plt.gca()) 
plt.title('Residuals')

plt.tight_layout()

There is a trend to increase the number of orders per day, orders do not depend on seasonality

In [ ]:
plt.figure(figsize=(20, 20))
plt.subplot(311)

decomposed.trend['2018-03-01':'2018-04-01'].plot(ax=plt.gca())
plt.title('Trend')

plt.subplot(312)
decomposed.seasonal['2018-03-01':'2018-04-01'].plot(ax=plt.gca()) 
plt.title('Seasonality')

plt.subplot(313)
decomposed.resid['2018-03-01':'2018-04-01'].plot(ax=plt.gca()) 
plt.title('Residuals')

plt.tight_layout()

## Conclusion

The data has been analyzed.
<br>Load schedules are similar during the week.
<br>There is a trend to increase the number of orders per day over time, the number of orders does not depend on the season

# Building forecast models for orders for the next hour

## Preparing data for models

Define a function that adds new features

In [ ]:
def make_features(data, max_lag, rolling_mean_size):
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['num_orders'].shift(lag)

    data['rolling_mean'] = data['num_orders'].shift().rolling(rolling_mean_size).mean()

Let's add new features

In [ ]:
data = copy.deepcopy(df)
make_features(data, 20, 2)
data.info()

Remove NaN values

In [ ]:
data.dropna(inplace=True)
data.info()

Divide the sample into sets with features and a target feature

In [ ]:
features = data.drop(['num_orders'],axis=1)
target = data['num_orders']

Let's break the sets into train and test in the ratio of `9:1`

In [ ]:
# features_train, features_test = train_test_split(features, test_size=0.1, shuffle=False)
# target_train, target_test = train_test_split(target, test_size=0.1, shuffle=False)

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.1, shuffle=False)

print(features.shape)
print(target.shape)
print()

print(features_train.shape)
print(target_train.shape)
print()

print(features_test.shape)
print(target_test.shape)

## Linear regression

In [ ]:
%%time
model_l_r = LinearRegression()
model_l_r.fit(features_train, target_train)
predictions_train = model_l_r.predict(features_train)
rmse_train = mean_squared_error(predictions_train, target_train) ** 0.5
print('RMSE of a linear model on a train set:', rmse_train)

predictions_test = model_l_r.predict(features_test)
rmse_test = mean_squared_error(predictions_test, target_test) ** 0.5
print('RMSE of a linear model on a test set:', rmse_test)

## Decision tree

In [ ]:
%%time
param_grid = {'max_depth': range(1,100)}
tscv = TimeSeriesSplit(n_splits=5)

dtr = GridSearchCV(estimator=DecisionTreeRegressor(random_state=12345), param_grid=param_grid, cv=tscv)
dtr.fit(features_train, target_train)
dtr.best_params_

In [ ]:
predictions_train = dtr.predict(features_train)
rmse_train = mean_squared_error(predictions_train, target_train) ** 0.5
print('RMSE of the decision tree on the train set:', rmse_train)

predictions_test = dtr.predict(features_test)
rmse_test = mean_squared_error(predictions_test, target_test) ** 0.5
print('RMSE of the decision tree on the test set:', rmse_test)

## Random forest

In [ ]:
%%time
param_grid = {'n_estimators': range(1,50,2), 'max_depth': range(1,20,2)}
tscv = TimeSeriesSplit(n_splits=5)

rfr = GridSearchCV(estimator=RandomForestRegressor(random_state=12345), param_grid=param_grid, cv=tscv)
rfr.fit(features_train, target_train)
rfr.best_params_

In [ ]:
predictions_train = rfr.predict(features_train)
rmse_train = mean_squared_error(predictions_train, target_train) ** 0.5
print('RMSE of the random forest on the train set:', rmse_train)

predictions_test = rfr.predict(features_test)
rmse_test = mean_squared_error(predictions_test, target_test) ** 0.5
print('RMSE of a random forest on a test set:', rmse_test)

## LightGBM

In [ ]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.005,
    'verbose': 0,
    "max_depth": 20,
    "num_iterations": 100000,
    "n_estimators": 1000
}

In [ ]:
%%time
gbm = lgb.LGBMRegressor(**hyper_params)
gbm.fit(features_train, target_train, verbose=0)
gbm.best_score_

In [ ]:
predictions_train = gbm.predict(features_train)
rmse_train = mean_squared_error(predictions_train, target_train) ** 0.5
print('RMSE LightGBM on the train set:', rmse_train)

predictions_test = gbm.predict(features_test)
rmse_test = mean_squared_error(predictions_test, target_test) ** 0.5
print('RMSE LightGBM on test set:', rmse_test)

Let's look at the importance of LightGBM factors

In [ ]:
feature_importances = pd.DataFrame(gbm.feature_importances_,
                                   index = features.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
feature_importances

# Conclusion

As can be seen from the results on the test dataset, LightGBM takes first place, and random forest is in second place.
<br>It should also be taken into account that LightGBM works, in this case, 3 times faster than random forest.
<br>It is worth noting that in order to get `rmse` less than 48, several iterations were carried out with the regulation of not only the hyperparameters of the models, but also, what is especially important, with the regulation of the number of differences in the time series and the size of the averaging window